In [ ]:
!pip install requests beautifulsoup4 pandas

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime

# Header giả lập trình duyệt để tránh bị chặn
headers = {"User-Agent": "Mozilla/5.0"}

# Danh sách để lưu dữ liệu từ nhiều trang
all_data = []

# Lặp qua các trang từ 2 đến 68
for page in range(2, 68):
    url = f"https://thanhnien.vn/timelinelist/18550/{page}.htm"

    try:
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            articles = soup.find_all("div", class_="box-category-item")

            for article in articles:
                title_tag = article.find("h3", class_="box-title-text")
                title = title_tag.get_text(strip=True) if title_tag else "Không có tiêu đề"

                link_tag = title_tag.find("a") if title_tag else None
                link = f"https://thanhnien.vn{link_tag['href']}" if link_tag else "Không có link"

                all_data.append({"Title": title, "Link": link})

        time.sleep(1)  # Tránh bị chặn do gửi quá nhiều request liên tục

    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu từ {url}: {e}")

# Danh sách để lưu dữ liệu chi tiết từ từng bài viết
detailed_data = []

# Truy cập từng bài viết để lấy thời gian đăng bài
for article in all_data:
    article_url = article["Link"]

    try:
        article_response = requests.get(article_url, headers=headers)

        if article_response.status_code == 200:
            article_soup = BeautifulSoup(article_response.text, "html.parser")
            time_tag = article_soup.find("div", {"data-role": "publishdate"})
            publish_time = time_tag.get_text(strip=True) if time_tag else "Không có thời gian"

            # Chuyển đổi định dạng ngày tháng từ dd/mm/yyyy -> dd-mm-yyyy
            try:
                formatted_date = datetime.strptime(publish_time.split()[0], "%d/%m/%Y").strftime("%d-%m-%Y")
            except:
                formatted_date = "Không có thời gian"

            detailed_data.append({
                "Title": article["Title"],
                "Link": article["Link"],
                "Publish Time": formatted_date
            })

        time.sleep(1)

    except Exception as e:
        print(f"Lỗi khi lấy dữ liệu từ {article_url}: {e}")

# Chuyển dữ liệu thành DataFrame
df_detailed = pd.DataFrame(detailed_data)

# Xuất dữ liệu ra file CSV
df_detailed.to_csv("thanhnien_news_pages_2_to_68.csv", index=False, encoding="utf-8-sig")

print("Dữ liệu đã được lưu vào thanhnien_news_pages_2_to_68.csv")


Dữ liệu đã được lưu vào thanhnien_news_pages_2_to_6.csv


In [ ]:
import pandas as pd

# Đọc dữ liệu từ file CSV
file_path = "thanhnien_news_pages_2_to_6.csv"
df = pd.read_csv(file_path, encoding="utf-8-sig")

# Chuyển cột "Publish Time" về kiểu datetime, bỏ giá trị không hợp lệ
df["Publish Time"] = pd.to_datetime(df["Publish Time"], format="%d-%m-%Y", errors="coerce")

# Lọc dữ liệu từ 29-09-2019 đến 30-09-2024
start_date = "29-09-2019"
end_date = "31-12-2024"
df_filtered = df[(df["Publish Time"] >= start_date) & (df["Publish Time"] <= end_date)]

# Xuất dữ liệu đã lọc ra file CSV mới
filtered_file_path = "thanhnien_news_filtered_1.csv"
df_filtered.to_csv(filtered_file_path, index=False, encoding="utf-8-sig")

print(f"Dữ liệu đã được lưu vào {filtered_file_path}")

Dữ liệu đã được lưu vào thanhnien_news_filtered_1.csv


In [ ]:
import pandas as pd

# Đọc dữ liệu từ file CSV đã tải lên
# Changed filename to the correct one created in previous step
df = pd.read_csv("/content/thanhnien_news_filtered_1.csv", encoding="utf-8-sig")

# Chuyển cột "Publish Time" về kiểu datetime, với định dạng ngày tháng dd-mm-yyyy
df["Publish Time"] = pd.to_datetime(df["Publish Time"]).dt.strftime("%d-%m-%Y")

# Lưu dữ liệu đã chuyển đổi ra file CSV mới
df.to_csv("thanhnien_news_filtered_1_final.csv", index=False, encoding="utf-8-sig")

print("Cột Publish Time đã được chuyển đổi về định dạng dd-mm-yyyy và lưu vào file thanhnien_news_filtered_1_final.csv")

Cột Publish Time đã được chuyển đổi về định dạng dd-mm-yyyy và lưu vào file thanhnien_news_filtered_1_final.csv
